In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()
import pyspark.sql.functions as spfn
# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [3]:
sc = spark.sparkContext

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


**Reading text file**

This creates a RDD

In [5]:
textfile=sc.textFile("/content/drive/MyDrive/news.txt")
print ("Total number of documents (each news represent a document ):",len(textfile.collect()))


Total number of documents (each news represent a document ): 12


In [6]:
def split(x):

  words=x.lower().split(" ")
  return words
words_in_docs=textfile.map(split)#returns a rdd of 12 elements, each element is a list of words in the corresponding document
print ("outputs of word splt map function :",len(words_in_docs.collect()))
print(type(words_in_docs.collect()[0]))

outputs of word splt map function : 12
<class 'list'>


**Computing term frequency(tf) for specified 3 words**

In [7]:
list_of_rdds=[]
rdd_of_tf=sc.parallelize([("market",0),("gas",0),("japan",0)])
lst=[]
def tf(x):
  lst.append(total)
  t=x[1]/total
  return x[0],x[1]/total
def check (z):
  if z[0] in ["japan","gas","market"]:
    return True
  else :
    return False

for j in words_in_docs.collect():
  tmp=sc.parallelize(j)
  total=tmp.count()
  count=tmp.map(lambda word: (word, 1)).reduceByKey(lambda z, y : z+y).filter(check)
  count=count.union(rdd_of_tf).reduceByKey(lambda z, y : z+y)
  for j in count.collect():
    lst.append((j[1]/total))
  count=count.map(lambda x:(x[0],x[1]/total))
  count.collect()#we need to run some method to force rdd to execute, otherwise we always get the total value of last rdd
  list_of_rdds.append(count)
print (lst)#just something to check
for j in list_of_rdds:
  print (j.collect())

[0.00966183574879227, 0.00966183574879227, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008, 0.0, 0.0, 0.0, 0.028846153846153848, 0.0, 0.0, 0.027950310559006212, 0.009316770186335404, 0.0, 0.018867924528301886, 0.018867924528301886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003703703703703704, 0.0, 0.0, 0.009433962264150943]
[('gas', 0.00966183574879227), ('japan', 0.00966183574879227), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.0), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.0), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.0), ('market', 0.0)]
[('gas', 0.008), ('japan', 0.0), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.028846153846153848), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.027950310559006212), ('market', 0.009316770186335404)]
[('gas', 0.0), ('japan', 0.018867924528301886), ('market', 0.018867924528301886)]
[('gas', 0.0), ('japan', 0.0), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.0), ('market', 0.0)]
[('gas', 0.0), ('japan', 0.0), ('market', 0.003703703703703704)]
[(

**computing inverse document frequency(idf)**

In [8]:
import math
number_of_docs=textfile.count()
def count_japan (x):
  x1=0
  if "japan" in x :
    x1=1
  return ("japan",x1)
def count_gas (x):
  x1=0
  if "gas" in x :
    x1=1
  return ("gas",x1)
def count_market (x):
  x1=0
  if "market" in x :
    x1=1
  return ("market",x1)
def idf (x):
  return (x[0],math.log(number_of_docs/x[1]))
df_japan=words_in_docs.map(count_japan)
df_gas=words_in_docs.map(count_gas)
df_market=words_in_docs.map(count_market)
idf_rdd=df_japan.union(df_gas.union(df_market))
idf_rdd=idf_rdd.reduceByKey(lambda a,b:a+b).map(idf)
print ("inverse document frequency for each word :",idf_rdd.collect())

inverse document frequency for each word : [('japan', 1.0986122886681098), ('gas', 1.791759469228055), ('market', 1.0986122886681098)]


**final step of tf-idf**

In [9]:
list_of_tf_idf=[]
for j in list_of_rdds:
  tf_idf=idf_rdd.union(j)
  tf_idf=tf_idf.reduceByKey(lambda x,y:x*y)
  tf_idf=tf_idf.map(lambda x: x[1])
  tf_idf.collect()
  list_of_tf_idf.append(tf_idf)
print ("market,gas,japan")
for i in list_of_tf_idf:
  print (i.collect())

market,gas,japan
[0.0, 0.017311685693024683, 0.010614611484716036]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.01433407575382444, 0.0]
[0.0, 0.0, 0.03169073909619548]
[0.01023551821740475, 0.0, 0.03070655465221425]
[0.0207285337484549, 0.0, 0.0207285337484549]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.004068934402474481, 0.0, 0.0]
[0.01036426687422745, 0.0, 0.0]


**Getting the argmax in order to find related document to each word**

In [11]:
j=list_of_tf_idf[0].collect()
market=sc.parallelize([j[0]])
gas= sc.parallelize([j[1]])
japan = sc.parallelize([j[2]])
for rdd in list_of_tf_idf[1:]:
  j=rdd.collect()
  market=market.union(sc.parallelize([j[0]]))
  gas=gas.union(sc.parallelize([j[1]]))
  japan=japan.union(sc.parallelize([j[2]]))
max_value_index_gas =[]
max_value_index_market = []
max_value_index_japan = []
m=market.zipWithIndex().takeOrdered(3,key=lambda x: -x[0])
g=gas.zipWithIndex().takeOrdered(3,key=lambda x: -x[0])
j=japan.zipWithIndex().takeOrdered(3,key=lambda x: -x[0])
"""  max_value_index_market.append(m)
  max_value_index_gas.append(g)
  max_value_index_japan.append(j)"""

'  max_value_index_market.append(m)\n  max_value_index_gas.append(g)\n  max_value_index_japan.append(j)'

In [12]:
print ("corresponding documents for each word")
print ("gas:",g)
print ("market:",m)
print ("japan:",j)

corresponding documents for each word
gas: [(0.017311685693024683, 0), (0.01433407575382444, 4), (0.0, 1)]
market: [(0.0207285337484549, 7), (0.01036426687422745, 11), (0.01023551821740475, 6)]
japan: [(0.03169073909619548, 5), (0.03070655465221425, 6), (0.0207285337484549, 7)]
